In [ ]:
source("../hub/map.r")
source("../hub/shortcuts.r")
source("../hub/run.r")

### 0 - Read prepped cohorts data

In [ ]:
ready <- readRDS("biomarker_cohorts2.Rds")

### 1 - Run across features

In [ ]:
features <- ready$features[1:3]

In [ ]:
mechanisms <- c("exact", "contains", "tissue_exact", "tissue_contains")

In [ ]:
df <- ready$exact$`Multikinase inhibitor`

In [ ]:
mod_map[['nrBor']] <- "bor"
mod_map[['nrDcb']] <- "bor"

In [ ]:
out <- data.frame()
n <- nrow(df)   
   
for( k in c("nrBor", "nrDcb")){    
out <- 
rbind(out, scanner(y = k, features, covariates = "+ as.factor(location)", df = "df", mod = mod_map[[k]] ) 
 %>% mu(type = i, mechanism = j, tissue = "all", n_samples = n))
}

In [ ]:
scanner <- function (y = "Surv(Y_os_days, Y_os_event)", features, covariates, 
    df = "df", mod = "coxph") 
{
    out <- data.frame()
    for (f in features) {
        if (grepl("rna_", f)) {
            tmp <- get_stats(y = y, x = f, covariate = paste0("+purity_purity", 
                covariates), data = df, model = mod)
        }
        else {
            tmp <- get_stats(y = y, x = f, covariate = covariates, 
                data = df, model = mod)
        }
        if (is.data.frame(tmp)) 
            out <- rbind(out, tmp)
    }
    out
}

### 2 - Long run

In [ ]:
out <- data.frame()
system.time(
for( i in mechanisms ){
 print(i); flush.console()

 if( !grepl("tissue", i)){
  cohorts <- ready[[i]]
    
  for( j in names(cohorts)){
   df <- cohorts[[j]]
   n <- nrow(df)   
   print(j); flush.console()
   
  for( k in c("nr_bor", "nr_dcb")){    
  out <- 
  rbind(out, scanner(y = k, features, covariates = "+ as.factor(location)", df = "df", mod = mod_map[[k]] ) 
     %>% mu(type = i, mechanism = j, tissue = "all", n_samples = n))
 }}} else {
  cohorts <- ready[[i]]   
  for(j in names(cohorts)){
    dfs <- cohorts[[j]]
    for( l in names(dfs)){
     df <- dfs[[l]]
     n <- nrow(df)    
    
    for( k in names(mod_map)){    
    out <- 
    rbind(out, scanner(y = k, features, covariates = "", df = "df", mod = mod_map[[k]] ) 
     %>% mu(type = i, mechanism = j, tissue = l, n_samples = n))
   }}}}
})

In [ ]:
table(out$type)

### 3 - Send it!

In [ ]:
#fwrite(out, "1_run_output.csv")
fwrite(out, "1_run_non_response_output.csv")

In [ ]:
go <- fread("1_run_output.csv")

In [ ]:
#go %>% fi(y == "durableClinicalBenefit") %>% ar(pval)

In [ ]:
see <- fread("1_run_non_response_output.csv")

In [ ]:
see %>% ar(pval)